In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import plot
from dtaidistance import dtw_ndim

In [142]:
import pandas as pd
import plotly
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

# def plot_dbscan(df, countries, labels):
data = pd.read_csv('data/data.csv')

# transform input data into adequate structure - 3D numpy array
data_t = data.melt(id_vars=['countrycode', 'country', 'year'])
data_t = data_t.groupby(['countrycode', 'country', 'year', 'variable'])['value'].aggregate('mean').unstack(
    'year')
data_t = data_t.reset_index().drop('variable', axis=1).groupby(['countrycode', 'country']).agg(list)
n_countries = data_t.shape[0]  # number of points (countries)
time_range = data_t.shape[1]  # time range
n_vars = data.shape[1] - 3  # number of economic indexes
# filling the array
data_t_arr = np.empty(shape=(n_countries, time_range, n_vars))
for i in range(n_countries):
    for j in range(time_range):
        data_t_arr[i][j] = np.array(data_t.iloc[i, j])
# calculating distances between points (countries)
dtw_matrix = dtw_ndim.distance_matrix(data_t_arr, n_vars)

# labels = labels.astype(str)
# countries["cluster"] = pd.Series(labels)
# data = [dict(type='choropleth',
#             locations = countries['countrycode'].astype(str),
#             color=countries['cluster'].astype(int),
#             projection='conic conformal', color_discrete_sequence=px.colors.qualitative.Pastel)]
countries = data[['countrycode','country']].drop_duplicates().reset_index(drop=True)
# let's create some additional, random data 

##### WYKRES ######
eps_grid = [3, 3.1, 3.2, 3.3, 3.4, 3.5]
#eps_grid  = [i for i in range(2, 6, 0.1)]
min_samples_grid = [2,3,4,5,6]
#min_samples_grid = [i for i in range(2,11)]
plot_data = []
for eps in eps_grid:
    for min_samples in min_samples_grid:
        model = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
        model.fit(dtw_matrix)
        plot_data.append(dict(type='choropleth',
                locations = countries['countrycode'].astype(str),
                z=model.labels_.astype(str),
                colorscale= [[0, '#718355'], [0.33, '#ffe8d6'], [0.6, '#ddbea9'], [1, '#cb997e']],
                showscale=False
))
# (3,2), (3,3) ... (3.1, 2), (3.1, 3) 

# let's create the steps for the slider
# eps
steps = []
i=0
for eps in eps_grid:
    for min_samples in min_samples_grid:
        step = dict(method='restyle',
                    args=['visible', [False] * len(plot_data)],
                    label='Eps: {} | Min samples: {}'.format(eps, min_samples))
        step['args'][1][i] = True
        steps.append(step)
        i+=1
# min_samples
# steps2 = []
# for i in range(len(plot_data)):
#     step = dict(method='restyle',
#                 args=['visible', [False] * len(plot_data)],
#                 label='Eps2 {}'.format(3+ i/10))
#     step['args'][1][i] = True
#     steps2.append(step)

sliders = [dict(active=0,
                steps=steps,
                currentvalue={'prefix':'Eps - ',
                'suffix':' - min samples'},
                len=0.8,
                xanchor='center',
                pad={"l":20,"r":20, "t":1},
                ticklen=10,
                x=0.5
                )]
                # dict(active=0,
                # pad={"t": 1},
                # steps=steps2)]    
layout = dict(geo=dict(projection={'type': 'conic conformal'}, lataxis={'range':[35,75]}, 
 lonaxis={'range' : [-15, 45]}),
            sliders=sliders)

fig = dict(data=plot_data, 
        layout=layout)
#fig.update_layout(showlegend=False)


In [143]:
plot(fig, filename='dbscan.html')

'dbscan.html'

In [55]:
data = pd.read_csv('data/data.csv')

# transform input data into adequate structure - 3D numpy array
data_t = data.melt(id_vars=['countrycode', 'country', 'year'])
data_t = data_t.groupby(['countrycode', 'country', 'year', 'variable'])['value'].aggregate('mean').unstack('year')
data_t = data_t.reset_index().drop('variable', axis=1).groupby(['countrycode', 'country']).agg(list)
n_countries = data_t.shape[0]  # number of points (countries)
time_range = data_t.shape[1]  # time range
n_vars = data.shape[1] - 3  # number of economic indexes
# filling the array
data_t_arr = np.empty(shape=(n_countries, time_range, n_vars))
for i in range(n_countries):
    for j in range(time_range):
        data_t_arr[i][j] = np.array(data_t.iloc[i, j])
# calculating distances between points (countries)
dtw_matrix = dtw_ndim.distance_matrix(data_t_arr, n_vars)

countries = data[['countrycode','country']].drop_duplicates().reset_index(drop=True)

##### WYKRES ######
eps_grid = [3, 3.1, 3.2, 3.3, 3.4, 3.5]
min_samples_grid = [2,3,4,5,6]
plot_data = []
for eps in eps_grid:
    for min_samples in min_samples_grid:
        model = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
        model.fit(dtw_matrix)
        plot_data.append(dict(type='choropleth',
                locations = countries['countrycode'].astype(str),
                z=model.labels_,
                customdata = [eps, min_samples]))

In [6]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import GridspecLayout
import ipywidgets as widgets

In [48]:
# Creates dropdown widget for eps
eps = widgets.Dropdown(
    options=[3, 3.1, 3.2, 3.3, 3.4, 3.5],
    value=3.1,
    description='Eps:',
    disabled=False,
    width='100px'
)

# Creates dropdown widget for min_samples
min_samples = widgets.Dropdown(
    options=[2,3,4,5,6], #np.arange(min_price,max_price+1000,1000),
    value=6, #np.arange(min_price,max_price+1000,1000)[-1],
    description='Min samples:',
    disabled=False,
    width='100px'
)

# Create container
container = widgets.VBox([widgets.HTML(value="<b>Params</b>"),eps, min_samples],justify_content='center',max_width='300px')                            

# Create grid
grid = GridspecLayout(1, 5, width='900px', height='150px',grid_gap="10px")

grid[0,0] = container

In [49]:
layout = dict(geo=dict(projection={'type': 'conic conformal'}, lataxis={'range':[35,75]}, 
 lonaxis={'range' : [-15, 45]}))

fig = go.FigureWidget(dict(data=plot_data, 
        layout=layout))

In [59]:
# This function will be called whenever there is any change in any of the widgets
def response(change):
    
    # Store indices of the cars within the price range
    eps_filter = list(np.where([x['customdata'][0] in [eps.value] for x in plot_data])[0])
    # Store indices of the brand
    print(eps_filter)
    min_samples_filter = list(np.where([x['customdata'][1] in [min_samples.value] for x in plot_data])[0])
    print(min_samples)
    # Store indices of the intersection of all the filters. 
    all_filters = list(set.intersection(set(eps_filter), set(min_samples_filter)))
    print(all_filters)
    # Set visibility for those cars at the intersection to true and rest to false
    visibility = [True if i in all_filters else False for i in range(len(plot_data))]
    
    # Update visibility of the plot so only the ones filtered will be true and rest will be false
    fig.plotly_restyle({'visible': visibility})
    
# link changes made in widget to the response function
eps.observe(response, names="value")
min_samples.observe(response, names="value")

In [60]:
# Plot grid of widgets as well as the figure
widgets.VBox([grid, fig])

[0, 1, 2, 3, 4]
Dropdown(description='Min samples:', index=4, options=(2, 3, 4, 5, 6), value=6)
[4]
[0, 1, 2, 3, 4]
Dropdown(description='Min samples:', index=3, options=(2, 3, 4, 5, 6), value=5)
[3]
[0, 1, 2, 3, 4]
Dropdown(description='Min samples:', index=1, options=(2, 3, 4, 5, 6), value=3)
[1]
[0, 1, 2, 3, 4]
Dropdown(description='Min samples:', options=(2, 3, 4, 5, 6), value=2)
[0]
[0, 1, 2, 3, 4]
Dropdown(description='Min samples:', index=3, options=(2, 3, 4, 5, 6), value=5)
[3]
[0, 1, 2, 3, 4]
Dropdown(description='Min samples:', index=2, options=(2, 3, 4, 5, 6), value=4)
[2]
[20, 21, 22, 23, 24]
Dropdown(description='Min samples:', index=2, options=(2, 3, 4, 5, 6), value=4)
[22]


In [80]:
loc_series = []
z_series = []
param_series = [] 
for i in range(len(plot_data)):
    loc_series.append(plot_data[i]['locations'])
    z_series.append(plot_data[i]['z'])
    param_series.append([str(plot_data[i]['customdata'][0])+" | "+str(plot_data[i]['customdata'][1])]*39)
loc_series = np.concatenate( loc_series, axis=0 )
z_series = np.concatenate( z_series, axis=0 )
param_series = np.concatenate( param_series, axis=0 )

In [89]:
df = pd.concat([pd.Series(loc_series), pd.Series(z_series), pd.Series(param_series)], axis=1)#, columns=['loc', 'z', 'p'])

In [91]:
df.columns = ['loc', 'z', 'p']